In [196]:
import pathlib
from pathlib import Path
from typing import List, Tuple, Union
import csv
import copy
import numpy as np
import matplotlib.pyplot as plt
import serpentTools
from serpentTools.parsers.results import ResultsReader
import click
import matplotlib.pyplot as plt

serpentTools.settings.rc["serpentVersion"] = "2.1.32"
P = 48 #max no of FA
Z = 6  #max no of slices

AV = 0.6221408E+24

# get atomic wt numbers, store in atomic_wt_dict
with open("isotope_awt_list.csv", mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        atomic_wt_dict = {rows[0]:rows[1] for rows in reader}


def discharge_bu() -> None:
    res_list = []

    with serpentTools.settings.rc:
        serpentTools.settings.rc["verbosity"] = "error"
    dep = serpentTools.read("wh_lfr_dep.m")
    
    return dep

dep = discharge_bu()


def split_name(name: str)->Tuple[int, int]:
    """The functions take a fuel name, separates the P and Z and takes out the indexes
        Note: It s made only for this specific nomeclature, needs modification if you 
        want more fancy names

    Args:
        name (str): Takes in a standard fuel position name "fuelPxxxZxxx"

    Returns:
        Tuple[int, int]: Gives out a tuple with the val of P and Z
    """
    without_fuel = name[4:]
    split_without_fuel  = without_fuel.split("Z")
    p_num = int(split_without_fuel[0][1:])
    z_num = int(split_without_fuel[1])
    return p_num, z_num

Executing <Task pending coro=<Kernel.dispatch_queue() running at /home/stalex/.pyenv/versions/3.7.13/lib/python3.7/site-packages/ipykernel/kernelbase.py:510> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f2cfbed32a0>()] created at /home/stalex/.pyenv/versions/3.7.13/lib/python3.7/site-packages/tornado/queues.py:248> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/stalex/.pyenv/versions/3.7.13/lib/python3.7/site-packages/tornado/ioloop.py:687] created at /home/stalex/.pyenv/versions/3.7.13/lib/python3.7/site-packages/tornado/gen.py:874> took 0.722 seconds


### NEW VERSION

In [97]:
#or material == "thorium" or material == "protactinium" or material == "uranium" or material == "neptunium" or material == "plutonium" or material == "americium" or material == "curium" or material == "lost":

In [ ]:
dep.materials["fuelP48Z1"].mdens[0:52]

In [182]:
materials = dep.materials.keys()
materials_dict = {}
# Go through each material index
for material in materials:
    if material == "total":
        # The continue, skips the loop interation at that point
        continue
    # Add the materials until 52 pos, exclude the total ones, big total and lost (maybe some way more intuitive)
    # Last column (last BU step in sss2) is taken and summed up for each fuel volume, store in dic wth key mdens_sum
    materials_dict[material] = {"mdens_sum": dep.materials[material].mdens[:, -1][0:52].sum(), "serpent_burnup": dep.materials[material].burnup[-1]}

In [187]:
print(dep.materials["fuelP48Z1"].mdens[:, -1][0:52])
#dep.materials["fuelP1Z1"].names[0:52]
print(materials_dict["fuelP48Z1"])

[5.94868e-16 2.04728e-10 1.04192e-11 4.61439e-09 1.66527e-13 5.08138e-09
 2.98547e-15 1.06652e-10 3.18360e-10 6.01613e-14 2.48395e-10 1.17512e-08
 2.50239e-08 3.40219e-04 3.13089e-03 1.13587e-02 8.90413e-06 7.34716e+00
 3.31465e-06 1.18090e-10 1.13972e-17 4.96735e-10 5.09236e-07 7.21094e-03
 2.43181e-06 4.79316e-04 2.08835e-08 7.77158e-08 6.78206e-03 9.66135e-01
 3.32114e-01 3.51202e-02 7.88212e-03 7.16213e-08 1.64443e-07 9.66471e-03
 1.05266e-06 1.92422e-04 9.17818e-04 3.52144e-09 2.26296e-09 1.05338e-12
 1.07979e-11 2.39429e-04 1.74229e-05 2.95015e-04 3.33464e-05 3.75490e-06
 2.51239e-07 2.39704e-08 2.47670e-14 1.36261e-15]
{'mdens_sum': 8.729094239329758, 'serpent_burnup': 3.69163}


In [191]:
for material in materials_dict:
    p_index, z_index = split_name(str(material))
    mdens_to_divide_key = f"fuelP1Z{z_index}"
    mdens_to_divide = dep.materials[mdens_to_divide_key].mdens[:, -1][0:52].sum()
    ratio = materials_dict[material]["mdens_sum"] / mdens_to_divide
    materials_dict[material]["mdens_ratio"] = ratio
    #print(ratio)

In [172]:
mdens_to_divide

11.216222304536812

In [193]:
for material in materials_dict.keys():
    p_index, z_index = split_name(material)
    burn_up = materials_dict[material]["serpent_burnup"]
    print(burn_up)
    if p_index == 1:
        materials_dict[material]["corrected_burnup"] = burn_up
        continue
    
    material_name = f"fuelP{p_index - 1}Z{z_index}"
    ratio = materials_dict[material_name]["mdens_ratio"]
    corrected_burnup = burn_up * ratio
    materials_dict[material]["corrected_burnup"] = corrected_burnup

3.69163
5.12255
6.08486
5.76139
5.29506
3.75304
6.31769
9.02688
10.7038
10.3267
9.13385
6.27521
5.04747
6.85439
8.05264
7.73275
7.09683
4.97981
3.67931
5.39228
6.22363
6.23382
5.20342
3.80893
5.99591
8.28688
9.44748
9.27485
8.25955
5.56558
7.75279
11.0771
13.1519
12.6848
11.3346
7.62157
9.28832
13.7348
16.2182
16.0579
13.4912
9.101
11.253
16.2738
18.6562
18.7518
16.0969
10.8194
8.67662
12.2453
14.3117
14.2823
11.8183
8.34917
5.86814
8.03496
10.0077
9.54769
8.35039
5.90039
11.0388
16.6417
19.9149
19.6734
16.4343
11.2276
13.5616
20.2713
24.033
24.3827
20.0143
13.6724
11.7657
18.2652
21.8996
21.4843
17.9649
11.8038
17.0808
27.0114
31.3698
31.8844
25.5275
17.1556
18.1819
28.5293
34.09
33.8195
28.3176
18.4373
16.5539
25.8199
31.6021
31.5921
25.7099
16.6755
14.0954
22.1733
26.7898
26.9272
21.5413
13.7788
13.2199
21.1081
26.1736
25.6058
21.2315
13.1112
10.7837
16.9822
21.1527
21.3903
17.0901
10.6885
9.78508
15.7169
19.7489
19.4645
15.6013
9.51849
7.10215
11.2835
13.9579
13.7753
10.6575
6.7735

In [178]:
average_burnups = {}
for material in materials_dict.keys():
    p_index, _ = split_name(material)
    average_burnup = 0
    for zidx in range(1 , Z + 1):
        key_construct = f"fuelP{p_index}Z{zidx}"
        average_burnup += materials_dict[key_construct]["corrected_burnup"] / Z
    average_burnups[f"P{p_index}"] = average_burnup
    

In [179]:
sum([value for key, value in average_burnups.items()])

294.58626293267275

In [180]:
average_burnups_axial = {}
for material in materials_dict.keys():
    _, z_index = split_name(material)
    average_axial_burnup = 0
    for pidx in range(1 , P + 1):
        key_construct = f"fuelP{pidx}Z{z_index}"
        average_axial_burnup += materials_dict[key_construct]["corrected_burnup"]
    average_burnups_axial[f"Z{z_index}"] = average_axial_burnup

sum([value for key, value in average_burnups_axial.items()]) / Z   

294.5862629326727